### ODE solvers

#### Objective:
   - comparing the Runge-Kutta and Euler-Cauchy approximation solutions to the actual solution of an ODE
   - visualize the effect of changing the step size on both methods solutions

#### How it works:
   - **Comparison plot:** the solution of the ordinary differential equation: $\frac{dy(t)}{dt}=cos(t)$ is plotted along with the approximations from both methods.
   - **Error plots**: highlighting the error in both methods in comparison to the ODE solution: $y(t) = sin(t)$
   - **$t_0$:** the starting point of array `t` (x-axis).
   - **$t_f$:** the ending point of array `t`.
   - **$h$:** the step size.

#### Discussion:
   - What do you notice when changing the step size $h$?
   - Based on the knowledge you gained from the session, do you think you can do such an exercise manually (with pen, paper and a calculator)? Which method would be less time consuming to solve?
   - What is the major downside of having a very small step size? 
   - Are there other methods that can yield better approximations for this ODE than the ones used here? Find yourself.

In [1]:
from bqplot import pyplot as plt
import bqplot as bqp

In [2]:
import numpy as np

In [3]:
import ipywidgets as widgets

In [4]:
# setting the initial conditions
t0 = 0
y0 = np.sin(t0)
tfinal = 1
h = 0.1

# setting the bqp figure objects

sc_x = bqp.LinearScale()
sc_y = bqp.LinearScale()
lineR = bqp.Lines(x=[], y=[], labels=['sin(t)'], colors=['darkblue'], display_legend = True, scales = {'x':sc_x, 'y': sc_y})
lineRK = bqp.Lines(x=[], y=[], labels=['RK approximation'], colors=['darkgreen'], display_legend = True, scales = {'x':sc_x, 'y': sc_y})
lineEu = bqp.Lines(x=[], y=[], labels=['Euler approximation'], colors=['magenta'], display_legend = True, scales = {'x':sc_x, 'y': sc_y})
ax_x = bqp.Axis(scale=sc_x, label='t')
ax_y = bqp.Axis(scale=sc_y, orientation='vertical', label='y(t)')
#mainfigLayout = widgets.Layout(width='1400px', height='600px')
fig = bqp.Figure(marks=[lineR, lineRK, lineEu], axes=[ax_x, ax_y], title='Comparison plot', 
                     legend_location='bottom', animation_duration=1000, preserve_aspect=True)
fig.layout.width = '700px'
fig.layout.length = '600px'

#display(fig)

RKerr_fig = plt.figure(animation_duration=1000, preserve_aspect=True, title='RK approximation - sin(t)')
RKerr_fig.layout.height = '300px'
RKerr_fig.layout.width = '400px'
RKerr_lin = plt.plot(x=[], y=[])

Eerr_fig = plt.figure(animation_duration=1000, preserve_aspect=True, title='Euler approximation - sin(t)')
Eerr_fig.layout.height = '300px'
Eerr_fig.layout.width = '400px'
Eerr_lin = plt.plot(x=[], y=[])

t0_var = widgets.FloatText(value=0, description=r'\(t_0\)')
tfinal_var = widgets.FloatText(value=3.14, step=3.14, description=r'\(t_f\)')
h_var = widgets.FloatText(value=0.1, step=0.1, description=r'\(h\)')

def update_plot(*args):
    t0 = t0_var.value
    tfinal = tfinal_var.value
    h = h_var.value
    
    y0 = np.sin(t0)
    def RungeKutta(F, t0, h, tfinal, y0):
        t = np.arange(t0,tfinal,h)
        yout = np.ones(t.size)
        yout[0] = y0

        for i in range(1,t.size):
            s1 = F(t[i-1])#, yout[i-1])
            s2 = F(t[i-1]+h/2)#, yout[i-1]+h*s1/2)
            s3 = F(t[i-1]+h/2)#, yout[i-1]+h*s2/2)
            s4 = F(t[i-1]+h)#, yout[i-1]+h*s3)
            yout[i] = yout[i-1] + h*(s1 + 2*s2 + 2*s3 + s4)/6
        return yout, t
    
    def Euler(F, t0, h, tfinal, y0):
        t = np.arange(t0,tfinal,h)
        tsize = t.size
        yout = np.ones(tsize)
        yout[0] = y0

        for i in range(1,tsize):
            s1 = F(t[i-1])#,yout[i-1])
            yout[i] = yout[i-1] + s1*h 
        return yout
    F = lambda t : np.cos(t)
    yout, t = RungeKutta(F, t0, h, tfinal, y0)
    youtE = Euler(F, t0, h, tfinal, y0)
    
    # update figure objects
    lineR.x, lineRK.x, lineEu.x = t, t, t
    lineR.y, lineRK.y, lineEu.y = np.sin(t), yout, youtE
    RKerr_lin.x, Eerr_lin.x = t, t 
    RKerr_lin.y = np.sin(t) - yout
    Eerr_lin.y = np.sin(t) - youtE

t0_var.observe(update_plot, names = 'value')
tfinal_var.observe(update_plot, names = 'value')
h_var.observe(update_plot, names = 'value')

right_box = widgets.VBox([RKerr_fig, Eerr_fig])
left_box = widgets.VBox([fig, t0_var,tfinal_var,h_var])
display(widgets.HBox([left_box, right_box]))

# call function update_plot when changing the model parameters values
update_plot(None)